## TensorFlowの画像分類モデルを変換してOpenVINOで使う方法を学ぶ

このチュートリアルでは先ほどと同様、TensorFlowの画像分類モデルをOpenVINO IRモデルに変換する方法について学びます。

## 必要なライブラリのインポート

In [ ]:
import cv2
import numpy as np

import tensorflow as tf
import tensorflow_hub

import openvino as ov

from PIL import Image
from IPython.display import display

## TensorFlowの学習済みモデルの読み込み (ダウンロード)

学習済みresnet50モデルをTensorFlow hubからダウンロードします。

In [ ]:
tf_model = tensorflow_hub.load("https://www.kaggle.com/models/tensorflow/resnet-50/frameworks/TensorFlow2/variations/classification/versions/1")

## PyTorchモデルをOpenVINO IRモデルに変換し、ファイルとして保存する

`convert_model()` APIを使用することでモデルを変換することができます。

In [ ]:
ov_model = ov.convert_model(tf_model)
ov.save_model(ov_model, 'resnet50tf.xml')

## 保存したOpenVINO IRモデルファイルを読み込み、デバイス向けにコンパイルする  

先のPyTorchの場合、入力フォーマットが`NCHW` (1,3,224,224)だったのに対して、TensorFlowでは`NHWC` (1,224,224,3)になっていることに注意してください。必ずではありませんが、TensorFlowの場合、`NHWC`のチャネルファーストフォーマットが基本です。

In [ ]:
ov_model = ov.Core().read_model('resnet50tf.xml')
print(ov_model.inputs[0])
ov_model.reshape((1,224,224,3))
print(ov_model.inputs[0])
compiled_model = ov.compile_model(ov_model, device_name='CPU')
compiled_model

## 入力画像の読み込みとプリプロセス

In [ ]:
img = cv2.imread('beaver.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224,224))
display(Image.fromarray(img))
img = img.astype(np.float32) / 255.0
img = np.expand_dims(img, axis=0)
img.shape

## OpenVINOで推論実行

In [ ]:
result = compiled_model.infer_new_request(img)
result = result[0].ravel()
print(result.shape, result[:40])

## 推論結果の表示
正しい推論結果が出ていることを確認してください。

In [ ]:
with open('synset_words.txt') as f:
    imagenet_labels = [ line.rstrip().split(maxsplit=1)[1] for line in f.readlines() ]

indices = np.argsort(result)[::-1]

for index in indices[:5]:
    print(index, imagenet_labels[index], result[index])

## まとめ  

ここではTensorFlowのResnet50モデルをOpenVINO IRモデルに変換して利用する方法について学びました。  
PyTorchのケースと同様、`convert_model()` APIにフレームワーク形式のモデルオブジェクトを渡すだけなので簡単に変換できるのがわかったと思います。  

高度なモデルなどのケースでは`convert_model()` APIに様々なオプションを指定してモデルを変換することも可能です。

また、TensorFlowとPyTorchでデフォルトの入力データ形式が異なることも覚えておいて下さい。TensorFlowはチャネルファースト`NHWC`、PyTorchは`NCHW`がデフォルトです。OpenVINOはどちらの形式も扱えますが、デフォルトは`NCHW`です。